In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from malaya_speech.torch_model.vits.model_infer import SynthesizerTrn
from malaya_boilerplate.train.config import HParams
import json

2023-08-27 21:42:36.096203: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-27 21:42:36.266055: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-27 21:42:37.285961: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-27 21:42:37.286116: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [3]:
config = 'malay_vits/jordan.json'

In [4]:
with open(config) as fopen:
    hps = HParams(**json.load(fopen))

In [5]:
from malaya_speech.utils.text import TTS_SYMBOLS
import torch

In [6]:
model = SynthesizerTrn(len(TTS_SYMBOLS),
                                   hps.data.filter_length // 2 + 1,
                                   hps.train.segment_size // hps.data.hop_length,
                                   **hps.model)

In [33]:
!ls malay_vits/logs/jordan/G_*.pth

malay_vits/logs/jordan/G_0.pth	    malay_vits/logs/jordan/G_5000.pth
malay_vits/logs/jordan/G_10000.pth  malay_vits/logs/jordan/G_55000.pth
malay_vits/logs/jordan/G_15000.pth  malay_vits/logs/jordan/G_60000.pth
malay_vits/logs/jordan/G_20000.pth  malay_vits/logs/jordan/G_65000.pth
malay_vits/logs/jordan/G_25000.pth  malay_vits/logs/jordan/G_70000.pth
malay_vits/logs/jordan/G_30000.pth  malay_vits/logs/jordan/G_75000.pth
malay_vits/logs/jordan/G_35000.pth  malay_vits/logs/jordan/G_80000.pth
malay_vits/logs/jordan/G_40000.pth  malay_vits/logs/jordan/G_85000.pth
malay_vits/logs/jordan/G_45000.pth  malay_vits/logs/jordan/G_90000.pth
malay_vits/logs/jordan/G_50000.pth


In [34]:
pth = 'malay_vits/logs/jordan/G_90000.pth'

In [35]:
model.load_state_dict(torch.load(pth, map_location=torch.device('cpu'))['model'])

<All keys matched successfully>

In [36]:
_ = model.eval()

In [37]:
from malaya_speech.torch_model.vits.commons import intersperse
import malaya_speech

In [38]:
normalizer = malaya_speech.utils.text.TextIDS(pad_to = None, is_lower = False)

In [46]:
text = 'I want like to eat chicken and fish and sate and coffee, predict batch of strings with probability'

In [47]:
t, ids = normalizer.normalize(text)
t

'I want like to eat chicken and fish and sate and coffee , predict batch of strings with probability .'

In [48]:
ids = intersperse(ids, 0)

In [49]:
ids = torch.LongTensor(ids)
ids_lengths = torch.LongTensor([ids.size(0)])
ids = ids.unsqueeze(0)

In [50]:
audio = model.infer(
    ids,
    ids_lengths,
    noise_scale=0.66,
    noise_scale_w=0.66,
    length_scale=1.0,
)

In [51]:
audio = audio[0][0,0].detach().numpy()

In [52]:
import IPython.display as ipd
ipd.Audio(audio, rate = 22050)

In [30]:
torch.save(model.state_dict(), 'jordan.pth')

In [31]:
from malaya_boilerplate.huggingface import upload_dict

In [32]:
files_mapping = {'jordan.pth': 'model.pth',
                config: 'config.json'}
upload_dict(model = 'VITS-jordan', files_mapping = files_mapping, username = 'mesolitica')

409 Client Error: Conflict for url: https://huggingface.co/api/repos/create (Request ID: Root=1-64eb533e-3d132df049848b582c4b9087;1b2c2f53-9956-48b6-ab21-8d08dc487e44)

You already created this model repo


jordan.pth:   0%|          | 0.00/145M [00:00<?, ?B/s]